# Transposition of solar irradiance

In this tutorial you will learn how to calculate solar irradiance on a tilted surface using transposition models available in pvlib-python.

The first step is to import a few Python packages.

In [ ]:
# Install pvlib on Google Colab as this is not a standard package.
!pip install pvlib

In [2]:
import pvlib  # library for PV and solar calculations
import pandas as pd  # library for data analysis
import matplotlib.pyplot as plt  # library for plotting
import numpy as np  # library for math and linear algebra

## Step 1: Retrieve example dataset
The next step is to load the example dataset, which contains hourly solar irradiance data for DTU.

The dataset is a typical meteorological year (TMY), meaning it consists of the most average month (irradiance wise) for a 10-year period. The data was retrieved from PVGIS, which we will investigate in the next lecture.

In [4]:
file_url = 'https://raw.githubusercontent.com/AdamRJensen/solar-resource-course/refs/heads/main/data/example_annual_dataset.csv'

data = pd.read_csv(file_url, index_col=[0], parse_dates=[0])

data.head()

,ghi,dhi,dni
time(UTC),,,
1990-01-01 00:00:00+00:00,0.0,0.0,0.0
1990-01-01 01:00:00+00:00,0.0,0.0,0.0
1990-01-01 02:00:00+00:00,0.0,0.0,0.0
1990-01-01 03:00:00+00:00,0.0,0.0,0.0
1990-01-01 04:00:00+00:00,0.0,0.0,0.0


## Step 2: Calculate solar position

In [5]:
location = pvlib.location.Location(
    latitude=55.79,
    longitude=12.52,
)

solpos = location.get_solarposition(data.index)

## Step 3: Calculate extraterrestrial irradiance

In [6]:
data['dni_extra'] = pvlib.irradiance.get_extra_radiation(data.index)

## Step 3: Estimate in-plane irradiance

Estimate the in-plane irradiance of a tilted surface using the [Hay-Davies model](https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.irradiance.haydavies.html). You may assume that the surface is tilted 30 degrees from horizontal and faces south.

For convenience, you may use the [pvlib.irradiance.get_total_irradiance](https://pvlib-python.readthedocs.io/en/stable/reference/generated/pvlib.irradiance.get_total_irradiance.html) function.

In [ ]:
# Write your code here
transposed_irradiance = 

## Step X: Determine which orientation has the highest annual solar irradiance

Determine which orientation has the highest incident annual solar irradiance. You can either do this by manually trying different combinations, or you can write a for loop that calculates the in-plane irradiance for all combinations of tilt and azimuth (e.g., in 1-degree increments).

*If you are feeling Python savvy, you can also use an optimization search algorithm to avoid having to search the entire design space.*


In [ ]:
# write your code here